
* Shane Lindquist
* <span style="background-color: goldenrod
      ">sjl241@scarletmail.rutgers.edu</span>

Exercise-1
----------

##### Use the dataframes in the cell below. Combine the dataframes in a way to display the answers to the following questions:
##### * How many shares of S1 does Anne hold?
##### * How many shares of Yahoo does John hold?

In [8]:
import pandas as pd
import numpy as np
import datetime

dfInvestors = pd.DataFrame({'INo':['I1', 'I2', 'I3'],'IName':['Anne', 'John', 'Elizabeth'],'PhoneNo':['412-456-4567', '585-673-4572', '763-567-8953']})
dfSecurities =  pd.DataFrame({'SNo':['S1', 'S2', 'S3', 'S4', 'S5', 'S6'], 'SName':['Microsoft', 'Kodak', 'IBM','Xerox', 'Amazon', 'Yahoo'], 'Price':[78, 45, 68, 123, 34,78]})
dfHoldings = pd.DataFrame({'INo':['I1','I1','I1','I1','I2','I2','I3','I3','I3'],'SNo':['S1','S2','S3','S4','S2','S6','S1','S3','S4'],'Qty':[500, 2000, 90,200,917,324,1000,346,780]})

In [19]:
dfmerged = pd.merge(dfInvestors, dfHoldings, on='INo')
result = dfmerged.loc[
    (dfmerged["IName"] == "Anne") & (dfmerged["SNo"] == "S1"),
    "Qty"
]

print(f"Anne holds {result.iloc[0]} shares of S1")


Anne holds 500 shares of S1


In [26]:
dfmerged2 = pd.merge(dfmerged, dfSecurities, on='SNo')
result2 = dfmerged2.loc[
    (dfmerged2["IName"] == "John") & (dfmerged2["SName"] == "Yahoo"),
    "Qty"
]

print(f"John holds {result2.iloc[0]} shares of Yahoo")

John holds 324 shares of Yahoo


Exercise-2: Date stuff
----------------------------
##### Use the Movie Lens dataset for answering this question: How many movies were rated in the fourth quarter of 2000?

In [30]:
#Use the Movie Lens dataset posted on Canvas for this exercise.
Moviesdf = pd.read_csv("C:/Users/lindq/OneDrive/Desktop/MSBA degree/Big Data Analysis/ml1mdata2000.csv")
Moviesdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   user_id     1000196 non-null  float64
 1   movie_id    1000129 non-null  float64
 2   rating      1000209 non-null  int64  
 3   timestamp   1000209 non-null  int64  
 4   gender      1000209 non-null  object 
 5   age         1000143 non-null  float64
 6   occupation  1000154 non-null  float64
 7   zip         1000136 non-null  object 
 8   title       1000036 non-null  object 
 9   genres      1000116 non-null  object 
dtypes: float64(4), int64(2), object(4)
memory usage: 76.3+ MB


The 'timestamp' column is reading as an int, so we will need to translate it to a datetime object in order to measure the data within the fourth quarter.
we can also see there are some null values, as the Non-Null Count is inconsistand amongst the different columns.

In [33]:
#Count all null values
Moviesdf.isnull().sum()

user_id        13
movie_id       80
rating          0
timestamp       0
gender          0
age            66
occupation     55
zip            73
title         173
genres         93
dtype: int64

since there are no null values in 'timestamp' or 'rating' we will still be able to accurately count the amount of ratings. However since there are multiple missing movie ID's missing we may still misscount the amount of unique movies rated within the time period. I will first change the datatype of 'timestamp' then re-measure the null values within q4

In [173]:
#get the data from the timestamp
Moviesdf['date']=pd.to_datetime(Moviesdf['timestamp'],unit='s',errors='coerce')

#use DatetimeIndex() to extract year and month:
Moviesdf['year']=pd.DatetimeIndex(Moviesdf['date']).year
Moviesdf['month']=pd.DatetimeIndex(Moviesdf['date']).month
Moviesdf['month'].unique()

#create subset of data in q4
Q4df = Moviesdf.loc[
    (Moviesdf['year'] == 2000) & (Moviesdf['month'] == 10) | (Moviesdf['year'] == 2000) & (Moviesdf['month'] == 11) | (Moviesdf['year'] == 2000) & (Moviesdf['month'] == 12)
]

#recount null values in new dataset
Q4df.isnull().sum()

user_id             0
movie_id           13
rating              0
timestamp           0
gender              0
age                48
occupation         38
zip                33
title              85
genres             66
date                0
year                0
month               0
rating_category     0
dtype: int64

There are 14 missing movie ID's within Q4, one solution to fix these null values is to check for overlap in missing values between movieID and Title. if there are Title values in the observations with null movie_id values, we can fill in the blanks by matching the titles to their movie_id in other observations

In [174]:
#Finding the full observations of the 14 missing movie_id
Q4df.loc[Q4df['movie_id'].isnull()]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month,rating_category
2168,4950.0,NaN,3,973483834,M,25.0,16.0,55421,James and the Giant Peach (1996),Animation|Children's|Musical,2000-11-06 04:10:34,2000,11,Good
811519,3952.0,NaN,3,974595637,F,45.0,1.0,12449,Barefoot in the Park (1967),Comedy,2000-11-19 01:00:37,2000,11,Good
886858,305.0,NaN,1,976497690,F,18.0,0.0,55414,Needful Things (1993),Drama|Horror,2000-12-11 01:21:30,2000,12,Okay
886859,319.0,NaN,1,976420886,F,50.0,6.0,33436,Needful Things (1993),Drama|Horror,2000-12-10 04:01:26,2000,12,Okay
886860,352.0,NaN,4,976332985,M,18.0,4.0,60115,Needful Things (1993),Drama|Horror,2000-12-09 03:36:25,2000,12,Good
886861,390.0,NaN,1,976304360,M,25.0,4.0,55405,Needful Things (1993),Drama|Horror,2000-12-08 19:39:20,2000,12,Okay
886862,424.0,NaN,3,976308869,M,25.0,17.0,55112,Needful Things (1993),Drama|Horror,2000-12-08 20:54:29,2000,12,Good
886863,461.0,NaN,3,976229482,M,50.0,7.0,55075,Needful Things (1993),Drama|Horror,2000-12-07 22:51:22,2000,12,Good
886864,524.0,NaN,4,976170130,M,18.0,0.0,91320,Needful Things (1993),Drama|Horror,2000-12-07 06:22:10,2000,12,Good
886866,533.0,NaN,3,976219310,M,25.0,12.0,27514,Needful Things (1993),Drama|Horror,2000-12-07 20:01:50,2000,12,Good


In [175]:
#find the unique movie values
Missingtitles = Q4df.loc[Q4df['movie_id'].isnull()]['title'].unique()
Missingtitles = pd.DataFrame(Missingtitles, columns=['title'])
Missingtitles

,title
0,James and the Giant Peach (1996)
1,Barefoot in the Park (1967)
2,Needful Things (1993)
3,"McCullochs, The (1975)"
4,Stranger in the House (1997)


In [176]:
#merge Missingtitles to get their ID
resultdf = pd.merge(
    Missingtitles,
    Moviesdf[['movie_id','title']].drop_duplicates(),
    on='title',
    how='left'
)
resultdf

,title,movie_id
0,James and the Giant Peach (1996),661.0
1,James and the Giant Peach (1996),NaN
2,Barefoot in the Park (1967),2870.0
3,Barefoot in the Park (1967),NaN
4,Needful Things (1993),2120.0
5,Needful Things (1993),NaN
6,"McCullochs, The (1975)",NaN
7,Stranger in the House (1997),NaN
8,Stranger in the House (1997),1718.0


out of the unique titles within the q4 data, one movie is still missing an ID completely. to fix this, we can insert a new unique ID for now to get an accurate measure.

In [177]:
#checking original movie_id's to make sure we don't accedently duplicate any
Moviesdf['movie_id'].sort_values(ascending=False)

#with a max value of 3952 as a movie_id in the dataset, we can add the id 3953 without interfearing with the existing data
Q4df.loc[
    (Q4df['title'].str.lower() == 'mccullochs, the (1975)') & (Q4df['movie_id'].isna()),
    'movie_id'
] = 3953

#verifying the observation was corrected
Q4df.loc[(Q4df['title'].str.lower() == 'mccullochs, the (1975)')]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month,rating_category
999754,1470.0,3953.0,1,974836437,M,18.0,4.0,94118,"McCullochs, The (1975)",Drama,2000-11-21 19:53:57,2000,11,Okay


In [178]:
#dropping null values from resultsdf
resultdf = resultdf.dropna()

#creating a dictionary for mapping
mapping = dict(
    zip(resultdf['title'].str.lower().str.strip(), resultdf['movie_id'])
)

#mapping the missing movie_id values to the values in the results dataframe
Q4df.loc[:,'movie_id'] = Q4df['movie_id'].fillna(
    Q4df['title'].str.lower().str.strip().map(mapping)
)

#checking for any remaining null values in movie_id
Q4df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 446574 entries, 0 to 1000205
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   user_id          446574 non-null  float64       
 1   movie_id         446574 non-null  float64       
 2   rating           446574 non-null  int64         
 3   timestamp        446574 non-null  int64         
 4   gender           446574 non-null  object        
 5   age              446526 non-null  float64       
 6   occupation       446536 non-null  float64       
 7   zip              446541 non-null  object        
 8   title            446489 non-null  object        
 9   genres           446508 non-null  object        
 10  date             446574 non-null  datetime64[ns]
 11  year             446574 non-null  int32         
 12  month            446574 non-null  int32         
 13  rating_category  446574 non-null  category      
dtypes: category(1), datetime

With the null values corrected, we can get a more accurate answer for the excersize question.

In [179]:
movie_count = Q4df['movie_id'].nunique()
print(f"There are {movie_count} unique values in the movie_id column. Thus there are {movie_count} unique movies which received ratings in Q4.")

There are 3598 unique values in the movie_id column. Thus there are 3598 unique movies which received ratings in Q4.


Exercise - 3: Categorization
----------------------------
##### Insert a new column called "rating_category" into the dataframe. It is coded as following depending on the value of "movie rating": 1 & 2: Okay; 3 & 4: Good; 5: Excellent

In [180]:
Moviesdf['rating_category'] = pd.cut(
    Moviesdf['rating'],
    bins=[0, 2, 4, 5],                # bin edges
    labels=['Okay', 'Good', 'Excellent'],     # category labels
    include_lowest=True
)

Moviesdf.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,date,year,month,rating_category
0,1.0,1193.0,5,978300760,F,1.0,10.0,48067,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-31 22:12:40,2000,12,Excellent
1,2.0,1193.0,5,978298413,M,56.0,16.0,70072,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-31 21:33:33,2000,12,Excellent
2,12.0,1193.0,4,978220179,M,25.0,12.0,32793,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 23:49:39,2000,12,Good
3,15.0,1193.0,4,978199279,M,25.0,7.0,22903,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 18:01:19,2000,12,Good
4,17.0,1193.0,5,978158471,M,50.0,1.0,95350,One Flew Over the Cuckoo's Nest (1975),Drama,2000-12-30 06:41:11,2000,12,Excellent


Exercise - 4: Grouping and ranking
----------------------------------
##### Create a dataframe with this information based on the Movie Lens dataset: Movie ID, Average rating, Absolute rank, Percentile rank


In [187]:
#first adding a rating column with the average rating per movie id
Movie_avg = Moviesdf[['movie_id','rating']].groupby('movie_id').mean().reset_index()
Movie_avg

,movie_id,rating
0,1.0,4.146846
1,2.0,3.201141
2,3.0,3.016736
3,4.0,2.729412
4,5.0,3.006757
...,...,...
3700,3948.0,3.635731
3701,3949.0,4.115132
3702,3950.0,3.666667
3703,3951.0,3.900000


In [197]:
#Adding absolute rank
Movie_avg.sort_values(by='rating')
Movie_avg['rank'] = Movie_avg['rating'].rank(method='dense',ascending=False)

Movie_avg

,movie_id,rating,rank,pctrank
0,1.0,4.146846,179.0,0.945479
1,2.0,3.201141,1664.0,0.428880
2,3.0,3.016736,1915.0,0.341700
3,4.0,2.729412,2223.0,0.213765
4,5.0,3.006757,1928.0,0.338192
...,...,...,...,...
3700,3948.0,3.635731,962.0,0.685020
3701,3949.0,4.115132,207.0,0.937922
3702,3950.0,3.666667,911.0,0.703509
3703,3951.0,3.900000,496.0,0.840486


In [189]:
#Adding Percentile rank
Movie_avg['pctrank'] = Movie_avg['rating'].rank(pct=True)
Movie_avg

,movie_id,rating,rank,pctrank
0,1.0,4.146846,203.0,0.945479
1,2.0,3.201141,2117.0,0.428880
2,3.0,3.016736,2440.0,0.341700
3,4.0,2.729412,2914.0,0.213765
4,5.0,3.006757,2453.0,0.338192
...,...,...,...,...
3700,3948.0,3.635731,1168.0,0.685020
3701,3949.0,4.115132,231.0,0.937922
3702,3950.0,3.666667,1099.5,0.703509
3703,3951.0,3.900000,592.0,0.840486


In [198]:
Movie_avg.sort_values(by='rank')

,movie_id,rating,rank,pctrank
926,989.0,5.0,1.0,0.998785
744,787.0,5.0,1.0,0.998785
3010,3233.0,5.0,1.0,0.998785
3413,3656.0,5.0,1.0,0.998785
3151,3382.0,5.0,1.0,0.998785
...,...,...,...,...
2638,2845.0,1.0,2763.0,0.004318
1062,1142.0,1.0,2763.0,0.004318
3408,3651.0,1.0,2763.0,0.004318
137,142.0,1.0,2763.0,0.004318


Exercise - 5: Pivot/crosstabs
-----------------------------
##### Use the Movie Lens dataset, and show a table with the average ratings of movies in Zip codes that contain the term "007".

In [207]:
#first creating a table to query averaging the rating by zipcode
table = pd.pivot_table(Moviesdf, values='rating', index=['title', 'zip'], aggfunc=np.mean)
table

C:\Users\lindq\AppData\Local\Temp\ipykernel_28320\3176844640.py:1: FutureWarning: The provided callable <function mean at 0x000001CA928A5940> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  table = pd.pivot_table(Moviesdf, values='rating', index=['title', 'zip'], aggfunc=np.mean)


rating
title                  zip          
$1,000,000 Duck (1971) 17870     5.0
                       19119     4.0
                       24018     3.0
                       31401     4.0
                       33308     3.0
...                              ...
eXistenZ (1999)        98136     4.0
                       98144     3.0
                       98225     3.0
                       98502     5.0
                       99504     4.0

[872850 rows x 1 columns]

In [208]:
#querying the table for zip codes containing '007'
table.reset_index()[table.reset_index().zip.str.contains('007')]

,title,zip,rating
127,'Til There Was You (1997),60076,4.0
219,"'burbs, The (1989)",30075,3.0
481,...And Justice for All (1979),30076,3.0
959,10 Things I Hate About You (1999),60073,3.0
960,10 Things I Hate About You (1999),60076,5.0
...,...,...,...
871457,Young Guns II (1990),60076,3.5
871649,"Young Poisoner's Handbook, The (1995)",30076,4.0
871777,Young Sherlock Holmes (1985),30075,4.0
871778,Young Sherlock Holmes (1985),30076,4.0
